In [23]:
import pandas as pd
import numpy as np
import re

In [24]:
file_path = '/Users/diana/Desktop/isear-test.xlsx'
custom_headers = ['Emotions', 'Text']
df = pd.read_excel(file_path, skiprows=1, header=None, names=custom_headers)

conda install openpyxl

In [25]:
df.head ()

,Emotions,Text
0,joy,My first feeling of falling in love. Coming a...
1,joy,When I saw that the mark I had obtained in an ...
2,joy,When I learnt that I had been admitted to the ...
3,anger,In a hurry to get to lectures and the car woul...
4,guilt,"Not attending school, when am not sick - but ..."


In [26]:
total_count = df['Emotions'].value_counts().sum()
print (total_count)

1150


In [27]:
df['Emotions'].value_counts()

Emotions
shame      182
fear       168
anger      166
disgust    166
sadness    161
guilt      155
joy        152
Name: count, dtype: int64

In [28]:
emotions_set = list (set (df['Emotions'].values))

In [65]:
print (df['Text'].dtype)
df['Text'] = df['Text'].str.lower ()
text_column = df['Text']
#print ("here", text_column)
tokenized_text = text_column.str.split()
#print (tokenized_text)

vocab = set() 
for tokens_list in tokenized_text:
    for token in tokens_list:
        if r'[^\w\s]' in token:
            if re.search(r'[^\w\s]', token):
                token = re.sub(r'[^\w\s]+$', '', token)
                if token not in vocab:
                    vocab.add (token[-1])
        if token not in vocab:
            vocab.add(token)
#print (list(vocab))


object
here 0       my first feeling of falling in love.  coming a...
1       when i saw that the mark i had obtained in an ...
2       when i learnt that i had been admitted to the ...
3       in a hurry to get to lectures and the car woul...
4       not attending school, when  am not sick - but ...
                              ...                        
1145     once i saw some children hunting a cat to death.
1146                       being insulted by my roommate.
1147    i had a habit of wetting my bed when i was you...
1148     when i was selected to the university of zambia.
1149    when i learnt that i had been accepted at the ...
Name: Text, Length: 1150, dtype: object
0       [my, first, feeling, of, falling, in, love., c...
1       [when, i, saw, that, the, mark, i, had, obtain...
2       [when, i, learnt, that, i, had, been, admitted...
3       [in, a, hurry, to, get, to, lectures, and, the...
4       [not, attending, school,, when, am, not, sick,...
                    

TypeError: 'list' object is not callable

In [30]:
from math import log

collection = [token for instance in tokenized_text for token in instance]

#tf-idf

def calculate_tfidf(token, document):
    tf = document.count(token) / len(document)
    idf = log(len(tokenized_text) / collection.count(token))
    tfidf = tf * idf
    return tfidf

tfidf_scores_list = []

for instance in tokenized_text:
    for word in vocab:
        if word not in instance:
            tfidf_scores = {0}
            tfidf_scores_list.append(tfidf_scores)
        else:
            tfidf_scores = {calculate_tfidf(token, instance) for token in set(instance)}
            tfidf_scores_list.append(tfidf_scores)

print (tfidf_scores_list[0])

KeyboardInterrupt: 

In [31]:
conda install scikit-learn


Channels:
 - defaults
 - conda-forge
 - huggingface
 - pytorch
Platform: osx-64
Solving environment: done

## Package Plan ##

  environment location: /Users/diana/miniconda3/envs/newenv

  added / updated specs:
    - scikit-learn


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    joblib-1.2.0               |  py310hecd8cb5_0         402 KB
    scikit-learn-1.3.0         |  py310h3ea8b11_1         7.6 MB
    scipy-1.12.0               |  py310ha516a68_0        21.9 MB
    threadpoolctl-2.2.0        |     pyh0d69192_0          16 KB
    ------------------------------------------------------------
                                           Total:        30.0 MB

The following NEW packages will be INSTALLED:

  joblib             pkgs/main/osx-64::joblib-1.2.0-py310hecd8cb5_0 
  scikit-learn       pkgs/main/osx-64::scikit-learn-1.3.0-py310h3ea8b11_1 
  scipy              pkgs/main/osx-64::s

In [38]:
import sklearn


In [57]:
from sklearn.feature_extraction.text import TfidfVectorizer


tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the corpus to obtain the TF-IDF sparse matrix
tfidf_matrix_sparse = tfidf_vectorizer.fit_transform(sentences_list)

print (tfidf_matrix_sparse)

AttributeError: 'list' object has no attribute 'lower'

In [10]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

from torch.autograd import Variable

In [11]:
class Emotions (nn.Module):

    def __init__(self, input_size, hidden_1, output_size):

        super (Emotions, self).__init__()
        self.fc1 = nn.Linear (input_size, hidden_1)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear (hidden_1, output_size)
    def forward (self, x):
        out = self.fc1 (x)
        out = self.relu1 (out)
        out = self.fc2 (out)
        return out



In [12]:
model = Emotions (1150, 50, 7)

In [13]:
import torch.utils

batch_size = 50
train_loader = torch.utils.data.DataLoader (dataset = df, batch_size=batch_size, shuffle =True)